In [1]:
import pandas as pd
import numpy as np

In [2]:

# here i have corrected data , but i need extraction and corrected data

corrected=pd.read_csv('corrected.csv')

original_column= [column for column in corrected.columns[7:] if  not "corrected" in column.split("_")]
#extraction data 
extraction= pd.concat([corrected.iloc[:, :7], corrected[original_column]], axis=1)

In [3]:
len(extraction["document_id"].unique()) #355827

374652

In [4]:
ground_truth = pd.read_excel("GroundTruth_KCD.xlsx")


def match_with_ground_truth(ground, data):
    merged_data = pd.merge(data, ground[['document ID']], how='inner', left_on='document_id', right_on='document ID')
    # Dropping the extra column from the merge
    merged_data.drop(columns=['document ID'], inplace=True)
    return merged_data

In [5]:
#match with ground_truth data
original_filtered_ground = match_with_ground_truth(ground_truth, extraction)
corrected_filtered_ground = match_with_ground_truth(ground_truth, corrected)

In [6]:
##get accurance of each document id to get line item number
def get_accurance(data):
    # Get the occurrence of each document_id
    document_id_counts = data['document_id'].value_counts().reset_index()
    document_id_counts.columns = ['document_id', 'count']
    return document_id_counts


In [7]:
len(original_filtered_ground["document_id"].unique()) #201

201

In [8]:
## get document id for original and corrected data
original_document_id_counts= get_accurance(original_filtered_ground)
corrected_document_id_counts= get_accurance(corrected_filtered_ground)


In [9]:
def get_meta_data(data, extracted=True):
    data_collection = set()

    for _, row in data.iterrows():
        # there is no difference between original and corrected data
        organization = row['code']
        customer_erp_id = row['erp_id']
        document_id = row['document_id']

        if extracted:
            po_number = row['po_number_original']
            po_date = row['po_date_original']
            doc_type = row['doc_type_original']
            data_collection.add((organization, customer_erp_id, document_id, po_number, po_date, doc_type))
        else:
      
            #corrected
            po_number_c= row['po_number_corrected']
            po_date_c= row['po_date_corrected']
            doc_type_c= row['doc_type_corrected']
            data_collection.add((organization, customer_erp_id, document_id,po_number_c, po_date_c ,  doc_type_c))
    
    # Convert the set to a list and create a DataFrame with appropriate columns
    data_collection = list(data_collection)
    
    if extracted:
        columns = ['Organization', 'ERP Number', 'document ID', 'po number', 'po date', 'doc type']
    else:
        columns = ['Organization', 'ERP Number', 'document ID', 'po number', 'po date', 'doc type']
    
    data_copy = pd.DataFrame(data_collection, columns=columns)
    return data_copy





In [10]:
#contain original data
original_copy=get_meta_data(original_filtered_ground , extracted=True)
#contain original and corrected data
corrected_copy=get_meta_data(corrected_filtered_ground,extracted=False)   

In [11]:
def add_unit_quantity(id_counts, data_copy,data):
    # Create a dictionary for fast lookups
    doc_id_to_count = dict(zip(id_counts['document_id'], id_counts['count']))
    
    # Add number of line items
    data_copy['num_line_items'] = data_copy['document ID'].map(doc_id_to_count)
    
    # Add empty unit and quantity columns
    for i in range(1, 9):
        data_copy[f'li{i}_unit'] = np.nan
        data_copy[f'li{i}_qty'] = np.nan

    collection = {}
    for idx, row in data.iterrows():
        doc_id = row['document_id']
        if doc_id not in collection:
            collection[doc_id] = []
            #original data
        collection[doc_id].append((row['unit_original'], row['quantity_original']))

    # Update data with values from collection
    for idx, row in data_copy.iterrows():
        doc_id = row['document ID']
        if doc_id in collection:
            # get respective unit and quantity based on document id 
            units_and_quantities = collection[doc_id]
            for i in range(min(len(units_and_quantities), 8)):  # Ensure we don't exceed 8 line items
                data_copy.at[idx, f'li{i+1}_unit'] = units_and_quantities[i][0]
                data_copy.at[idx, f'li{i+1}_qty'] = units_and_quantities[i][1]

    return data_copy
        
    



In [12]:
def add_unit_quantity_non_original(id_counts, data_copy, data):

    ### Add num line item
    doc_id_to_count = dict(zip(id_counts['document_id'], id_counts['count']))
    
    data_copy['num_line_items'] = data_copy['document ID'].map(doc_id_to_count)
    
    # Add empty unit and quantity columns for both original and non-original
    for i in range(1, 9):
        data_copy[f'li{i}_unit'] = np.nan
        data_copy[f'li{i}_qty'] = np.nan

    ### Collect quantity and unit from the data itself
    collection = {}
    for idx, row in data.iterrows():
        doc_id = row['document_id']
        if doc_id not in collection:
            collection[doc_id] = []
            #corrected data
        collection[doc_id].append((row['unit_corrected'], row['quantity_corrected']))
       
    # Update data with values from collection
    for idx, row in data_copy.iterrows():
        doc_id = row['document ID']
        if doc_id in collection:
            # Get respective unit and quantity based on document id 
            units_and_quantities = collection[doc_id]
            for i in range(min(len(units_and_quantities), 8)):
                data_copy.at[idx, f'li{i+1}_unit'] = units_and_quantities[i][0]
                data_copy.at[idx, f'li{i+1}_qty'] = units_and_quantities[i][1]

    return data_copy

In [13]:
Final_original=add_unit_quantity(original_document_id_counts, original_copy,original_filtered_ground)
Final_corrected=add_unit_quantity_non_original(corrected_document_id_counts, corrected_copy,corrected_filtered_ground)

/var/folders/05/m0z3j2y1647d9pk4d3932ks80000gn/T/ipykernel_23845/1737071518.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'PCE' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data_copy.at[idx, f'li{i+1}_unit'] = units_and_quantities[i][0]
/var/folders/05/m0z3j2y1647d9pk4d3932ks80000gn/T/ipykernel_23845/1737071518.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'PCE' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data_copy.at[idx, f'li{i+1}_unit'] = units_and_quantities[i][0]
/var/folders/05/m0z3j2y1647d9pk4d3932ks80000gn/T/ipykernel_23845/1737071518.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'PCE' has dtype incompatible with float64, pl

In [14]:
Final_original["ERP Number"]=Final_original["ERP Number"].astype(int)
Final_corrected["ERP Number"]=Final_corrected["ERP Number"].astype(int)


###dtype
Final_original=Final_original.astype(str)
Final_corrected=Final_corrected.astype(str)
ground_truth=ground_truth.astype(str)

In [15]:
def compare(ground, extraction=None, correction=None, given_data_extracted=False, given_data_corrected=False, num_data_given=2):
    unmatched_document_id = []
    check_document_id=[]
    # Validate input parameters
    if not (given_data_extracted or given_data_corrected):
        raise ValueError("At least one of 'given_data_extracted' or 'given_data_corrected' must be given")

    if num_data_given not in [2, 3]:
        raise ValueError("'num_data_given' must be either 2 or 3")

    # Process each row in the ground truth data
    for i, row in ground.iterrows():
        document_id = row["document ID"]
        ground_truth_row = ground[ground["document ID"] == document_id].reset_index(drop=True)

        # Check against extracted data if provided
        if given_data_extracted and extraction is not None:
            extracted_row = extraction[extraction["document ID"] == document_id].reset_index(drop=True)
            #check between extracted data and ground truth 
            if ground_truth_row.equals(extracted_row):
                print(f"The extraction and ground truth completely match {document_id}")
                
            else:
                unmatched_document_id.append(document_id)

        # Check against corrected data if provided
        if given_data_corrected and correction is not None:
            corrected_row = correction[correction["document ID"] == document_id].reset_index(drop=True)
            #check between corrected data and ground truth
            if ground_truth_row.equals(corrected_row):
                print(f"The correction and ground truth completely match {document_id}")
            else:
                unmatched_document_id.append(document_id)

        # Additional check when both extracted and corrected data are given
        if num_data_given == 3 and given_data_extracted and given_data_corrected:
            extracted_row = extraction[extraction["document ID"] == document_id].reset_index(drop=True)
            corrected_row = correction[correction["document ID"] == document_id].reset_index(drop=True)
            #check between extracted data then not match with corrected data 
            if ground_truth_row.equals(extracted_row) and not ground_truth_row.equals(corrected_row):
                print(f"Business change detected for document ID {document_id}")
                check_document_id.append(document_id)
                

    return unmatched_document_id ,check_document_id


In [16]:
#extraction
EX_unmatch_doc,_ = compare(ground_truth, extraction=Final_original,correction=None, given_data_extracted=True, num_data_given=2)

The extraction and ground truth completely match 12d3653d-c7d8-4ed0-a65d-799318eb35c1
The extraction and ground truth completely match 14824f09-cde7-4c55-809e-f5cffe463b9f
The extraction and ground truth completely match 23120b32-0a2f-4be9-b41b-05f31db1dd08
The extraction and ground truth completely match 2742fb5f-be8e-405c-b7cb-847b0955a84d
The extraction and ground truth completely match 27e9108a-ef71-4dc3-82a9-fdb683f40505
The extraction and ground truth completely match 3606609d-8e96-426d-bfcf-b8cab8b0aaeb
The extraction and ground truth completely match 382fa376-b8f0-4ed5-941a-1dae1886167a
The extraction and ground truth completely match 4560c6ec-e317-4e58-9b46-d75220873337
The extraction and ground truth completely match 49757809-d024-43fc-a1d0-5eb4faa2e1b2
The extraction and ground truth completely match 773e7bd7-96d8-49a2-b1ae-702a04bc0cd9
The extraction and ground truth completely match 78b43585-9738-4212-b9c7-2c4a79ebc86d
The extraction and ground truth completely match 7f9b7

In [17]:
#business change
EX_unmatch_doc,ids_to_check =compare(ground_truth, extraction=Final_original,correction=Final_corrected, given_data_corrected=True,given_data_extracted=True, num_data_given=3)


The extraction and ground truth completely match 12d3653d-c7d8-4ed0-a65d-799318eb35c1
The correction and ground truth completely match 12d3653d-c7d8-4ed0-a65d-799318eb35c1
The extraction and ground truth completely match 14824f09-cde7-4c55-809e-f5cffe463b9f
The correction and ground truth completely match 14824f09-cde7-4c55-809e-f5cffe463b9f
The correction and ground truth completely match 21c1c444-b4d5-4b05-95da-2144da48612e
The correction and ground truth completely match 22adbf0f-bece-4bf8-9a28-7570b5029988
The extraction and ground truth completely match 23120b32-0a2f-4be9-b41b-05f31db1dd08
The correction and ground truth completely match 23120b32-0a2f-4be9-b41b-05f31db1dd08
The extraction and ground truth completely match 2742fb5f-be8e-405c-b7cb-847b0955a84d
Business change detected for document ID 2742fb5f-be8e-405c-b7cb-847b0955a84d
The extraction and ground truth completely match 27e9108a-ef71-4dc3-82a9-fdb683f40505
Business change detected for document ID 27e9108a-ef71-4dc3-82

In [18]:
#correction
COR_unmatch_doc,_= compare(ground_truth, correction=Final_corrected,extraction=None, given_data_corrected=True, num_data_given=2)

The correction and ground truth completely match 12d3653d-c7d8-4ed0-a65d-799318eb35c1
The correction and ground truth completely match 14824f09-cde7-4c55-809e-f5cffe463b9f
The correction and ground truth completely match 21c1c444-b4d5-4b05-95da-2144da48612e
The correction and ground truth completely match 22adbf0f-bece-4bf8-9a28-7570b5029988
The correction and ground truth completely match 23120b32-0a2f-4be9-b41b-05f31db1dd08
The correction and ground truth completely match 317ddee6-14e7-4498-8fad-a52f663f5a7d
The correction and ground truth completely match 363420be-2fe4-4f99-8306-fdec17861915
The correction and ground truth completely match 3bab17f3-a115-4160-9611-40d774a4defd
The correction and ground truth completely match 3be0ecc1-8ee9-4c57-baaa-12b1200de8e1
The correction and ground truth completely match 3f09d15e-55f7-4870-8dc8-e23393490788
The correction and ground truth completely match 4560c6ec-e317-4e58-9b46-d75220873337
The correction and ground truth completely match 466d9